In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import logging

import src
from src.bert.dataset import PBertDataset
from src.bert.dataset.strategies import MLMin1PopIdeol

In [2]:
logging.set_verbosity_error()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
THRESHOLDS = {0: 0.415961, 1: 0.295400, 2: 0.429109, 3: 0.302714}
print(DEVICE)

cuda


In [3]:
EXCLUDE_CODERS = []
test = PBertDataset.from_disk(
    path=src.PATH / "data/labeled_data/test.csv.zip",
    label_strategy=MLMin1PopIdeol(),
    exclude_coders=EXCLUDE_CODERS,
)

In [4]:
COMMIT_HASH = "cf44004e90045cde298e28605ff105747d58aa7a"

tokenizer = AutoTokenizer.from_pretrained("luerhard/PopBERT", revision=COMMIT_HASH)
model = AutoModelForSequenceClassification.from_pretrained(
    "luerhard/PopBERT", revision=COMMIT_HASH
).to(DEVICE)

In [5]:
collate_fn = test.create_collate_fn(tokenizer)
test_loader = DataLoader(test, collate_fn=collate_fn, batch_size=64, shuffle=False)

In [6]:
def apply_thresh(y_proba, thresholds: dict):
    y_proba = y_proba.copy()
    for dim, thresh in thresholds.items():
        y_proba[:, dim] = np.where(y_proba[:, dim] > thresh, 1, 0)
    return y_proba

In [7]:
with torch.inference_mode():
    y_true = []
    y_pred = []
    for batch in test_loader:
        encodings = batch["encodings"]
        encodings = encodings.to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        out = model(**encodings)
        preds = torch.nn.functional.sigmoid(out.logits)
        y_true.extend(batch["labels"].numpy())
        y_pred.extend(preds.cpu().numpy())
    y_pred_05 = np.where(np.array(y_pred) > 0.5, 1, 0)
    y_pred_thresh = apply_thresh(np.array(y_pred), THRESHOLDS)
    y_true = np.array(y_true)

In [8]:
print(
    classification_report(
        y_true,
        y_pred_05,
        target_names=["elite", "pplcentr", "left", "right"],
        zero_division=0,
    )
)

              precision    recall  f1-score   support

       elite       0.82      0.86      0.84       648
    pplcentr       0.77      0.60      0.67       322
        left       0.71      0.75      0.73       279
       right       0.77      0.57      0.65       155

   micro avg       0.78      0.75      0.76      1404
   macro avg       0.77      0.69      0.72      1404
weighted avg       0.78      0.75      0.76      1404
 samples avg       0.39      0.38      0.38      1404



In [9]:
print(
    classification_report(
        y_true,
        y_pred_thresh,
        target_names=["elite", "pplcentr", "left", "right"],
        zero_division=0,
    )
)

              precision    recall  f1-score   support

       elite       0.81      0.88      0.84       648
    pplcentr       0.70      0.73      0.71       322
        left       0.69      0.77      0.73       279
       right       0.68      0.66      0.67       155

   micro avg       0.75      0.80      0.77      1404
   macro avg       0.72      0.76      0.74      1404
weighted avg       0.75      0.80      0.77      1404
 samples avg       0.41      0.40      0.40      1404



In [10]:
performance = classification_report(
    y_true,
    y_pred_thresh,
    target_names=["elite", "pplcentr", "left", "right"],
    zero_division=0,
    output_dict=True,
)

In [11]:
out = pd.DataFrame(performance).T.drop("support", axis=1)
out = out.reset_index().rename(
    {"index": "Dimension", "precision": "Precision", "f1-score": "F1", "recall": "Recall"},
    axis=1,
)
out["Dimension"] = out["Dimension"].replace(
    {
        "elite": "Anti-Elitism",
        "pplcentr": "People-Centrism",
        "left": "Left-Wing Ideology",
        "right": "Right-Wing Ideology",
    }
)

out = out.iloc[:6, :].round(2)

In [12]:
out.to_latex(src.PATH / "results/tables/model_performance.tex", index=False)

/scratch/st_ac138201/job_2959149_o04c03/ipykernel_3454729/11744537.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  out.to_latex(src.PATH / "results/tables/model_performance.tex", index=False)
